In [1]:
import pandas as pd
import gensim

In [2]:
DATA_DIR = "../../data/raw/"
INPUT_FILE_NAME = 'cleaned.parquet'
OUTPUT_FILE_NAME = 'cleaned_squashed.parquet'

In [3]:
df = pd.read_parquet(DATA_DIR + INPUT_FILE_NAME)
df.head()

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"[thank, chris, truly, great, honor, opportunit...",thank chris truly great honor opportunity come...
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"[term, invention, like, tell, tale, favorite, ...",term invention like tell tale favorite project...
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"[public, dewey, long, ago, observe, constitute...",public dewey long ago observe constitute discu...
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"[want, start, say, houston, problem, enter, se...",want start say houston problem enter second ge...
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"[want, talk, background, idea, car, art, actua...",want talk background idea car art actually mea...


In [4]:
tags = df['tags'].str.replace(', ', ',').str.lower().str.strip()
split_tags = tags.str.split(',')
tag_counts_per_talk = split_tags.apply(len)

joined_tags = tags.str.cat(sep=',').split(',')
all_tags_w_dup = pd.Series(joined_tags)

tag_counts = all_tags_w_dup.value_counts()


In [6]:
tag_documents = tags.str.cat(sep='_').split('_')
def tokenise(documents):
    for string in documents:
        yield gensim.utils.simple_preprocess(string)
tag_documents = list(tokenise(tag_documents))

transcript_documents = df['clean_transcript'].map(lambda x: x.tolist())
transcript_documents = list(transcript_documents)

In [7]:
tag_model = gensim.models.Word2Vec(tag_documents,size=150,window=5,min_count=2,workers=4)
tag_model.train(tag_documents,total_examples=len(tag_documents),epochs=10)

transcript_model = gensim.models.Word2Vec(transcript_documents,size=150,window=5,min_count=2,workers=4)
transcript_model.train(transcript_documents,total_examples=len(transcript_documents),epochs=10)

(17856330, 19253440)

In [8]:
w1 = 'car'
# tag_model.wv.most_similar(positive=w1,topn=10)
transcript_model.wv.most_similar(positive=w1,topn=10)

[('selfdriving', 0.7736056447029114),
 ('driver', 0.7323495149612427),
 ('driverless', 0.6799625158309937),
 ('truck', 0.620323896408081),
 ('vehicle', 0.6158387660980225),
 ('bicycle', 0.6148133277893066),
 ('fuelefficient', 0.6087291240692139),
 ('cyclist', 0.6032378077507019),
 ('driveway', 0.5996245741844177),
 ('prius', 0.5948493480682373)]

In [ ]:
tags = df['tags'].str.replace(', ', ',').str.lower().str.strip()
split_tags = tags.str.split(',')
tag_counts_per_talk = split_tags.apply(len)

joined_tags = tags.str.cat(sep=',').split(',')
all_tags_w_dup = pd.Series(joined_tags)

tag_counts = all_tags_w_dup.value_counts()

# df['tags'] = split_tags

tag_cutoff = int(0.01*len(df.index))
print(tag_cutoff)
squashed_tags = pd.DataFrame(tag_counts)
# squashed_tags.columns = ['tag','count']
squashed_tags = squashed_tags[(squashed_tags[0]>tag_cutoff)]
squash_list = list(squashed_tags.index.values)
print(len(squash_list))

In [ ]:
for i in range(len(df.index)):
    original_tags = df['tags'][i]
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in squash_list:
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    df['tags'][i] = final_string

In [ ]:
# df[df['tags']=='']

In [ ]:
# df.to_pickle("./squashed_processed_data.pkl")

In [ ]:
# df['tags']